In [18]:
import cv2
import numpy as np
from tqdm import tqdm,trange
def resize_video(input_path, output_path, target_height, target_width):
    video = cv2.VideoCapture(input_path)
    fps = video.get(cv2.CAP_PROP_FPS)

    fourcc = cv2.VideoWriter_fourcc(*'avc1') 
    output_video = cv2.VideoWriter(output_path, fourcc, fps, (target_width, target_height))

    while True:
        ret, frame = video.read()
        if not ret:
            break

        resized_frame = cv2.resize(frame, (target_width, target_height))
        output_video.write(resized_frame)

    video.release()
    output_video.release()

def show_video_info(path):
    cap = cv2.VideoCapture(path)

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    fps = cap.get(cv2.CAP_PROP_FPS)
    cap.release()
    print("height:", height)
    print("width:", width)
    print("fps:", fps)
    print("total_frames:", total_frames)

def get_frames(path):
    cap = cv2.VideoCapture(path)
    height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    fps = cap.get(cv2.CAP_PROP_FPS)
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
    cap.release()
    return frames, int(height), width, fps

def save_frames(frames,height, width, fps, path):
    fourcc = cv2.VideoWriter_fourcc(*'avc1') 
    output_video = cv2.VideoWriter(path, fourcc, fps, (width, height))
    for frame in tqdm(frames):
        output_video.write(frame)
    output_video.release()

palette=np.array([[128,64,128],[244,35,232],[70,70,70],[102,102,156],
         [190,153,153],[153,153,153],[250,170,30],[220,220,0],
         [107,142,35],[152,251,152],[70,130,180],[220,20,60],
         [255,0,0],[0,0,142],[0,0,70],[0,60,100],[0,80,100],[0,0,230],[119,11,32]],dtype=np.uint8)

In [10]:
from tqdm import tqdm
from mmseg.apis import inference_model, init_model
frames, height, width, fps=get_frames('demo1.mp4')
model = init_model('config.py', 'checkpoint.pth', device='cuda:0')
rs=[]
for frame in tqdm(frames):
    result = inference_model(model, frame)
    rs.append(result.pred_sem_seg.data.cpu().numpy().squeeze())

converted_rs=[]
for r in tqdm(rs):
    converted_rs.append(palette[r])
save_frames(converted_rs, 1024, 2048, 30, 'rs1.mp4')

/home/yichuan/codes/mmseg/mmsegmentation/mmseg/models/builder.py:36: UserWarning: ``build_loss`` would be deprecated soon, please use ``mmseg.registry.MODELS.build()`` 
  warnings.warn('``build_loss`` would be deprecated soon, please use '


Loads checkpoint by local backend from path: ddrnet_23-slim_in1k-pre_2xb6-120k_cityscapes-1024x1024_20230426_145312-6a5e5174.pth


100%|██████████| 2899/2899 [00:59<00:00, 48.44it/s]


In [5]:
image_files = sorted([f for f in os.listdir('/home/yichuan/codes/mmseg/mmsegmentation/cityspace/leftImg8bit/demoVideo/stuttgart_00')])+\
sorted([f for f in os.listdir('/home/yichuan/codes/mmseg/mmsegmentation/cityspace/leftImg8bit/demoVideo/stuttgart_01')])+\
sorted([f for f in os.listdir('/home/yichuan/codes/mmseg/mmsegmentation/cityspace/leftImg8bit/demoVideo/stuttgart_02')])
sample_image = cv2.imread(os.path.join('/home/yichuan/codes/mmseg/mmsegmentation/cityspace/leftImg8bit/demoVideo/stuttgart_00', image_files[0]))
height, width, _ = sample_image.shape
fps=30
# 创建视频编码器
fourcc = cv2.VideoWriter_fourcc(*'avc1')
video_writer = cv2.VideoWriter('demo1.mp4', fourcc, fps, (width, height))

# 逐帧写入视频
for image_file in image_files:
    image_path = os.path.join(image_dir, image_file)
    frame = cv2.imread(image_path)
    video_writer.write(frame)

# 释放资源
video_writer.release()

In [7]:
from tqdm import tqdm
fps=30
fourcc = cv2.VideoWriter_fourcc(*'avc1')
save_path='demo1.mp4'
width=2048
height=1024
video_writer = cv2.VideoWriter(save_path, fourcc, fps, (width, height))
for input_dir in ['/home/yichuan/codes/mmseg/mmsegmentation/cityspace/leftImg8bit/demoVideo/stuttgart_00','/home/yichuan/codes/mmseg/mmsegmentation/cityspace/leftImg8bit/demoVideo/stuttgart_01','/home/yichuan/codes/mmseg/mmsegmentation/cityspace/leftImg8bit/demoVideo/stuttgart_02']:
    image_files = sorted([f for f in os.listdir(input_dir)])
    for image_file in tqdm(image_files):
        image_path = os.path.join(input_dir, image_file)
        frame = cv2.imread(image_path)
        video_writer.write(frame)
video_writer.release()

  0%|          | 0/599 [00:00<?, ?it/s]

100%|██████████| 1200/1200 [01:32<00:00, 12.96it/s]


In [14]:
frames1, height, width, fps=get_frames('demo1.mp4')
frames2, height, width, fps=get_frames('rs1.mp4')

In [19]:
frames=[]
for i in trange(len(frames1)):
    frames.append(np.concatenate([frames1[i],frames2[i]],axis=0))
save_frames(frames, 2048, 2048, 30, 'rs2.mp4')

100%|██████████| 2899/2899 [01:00<00:00, 48.06it/s]
